In [ ]:
### This is the first attempt at using our own IP by loading the base overlay to get HDMI access
### then loading our custom overlay to process the captured frame
### then loading the base overlay again to send the processed frame via HDMI out
### it is very inefficient so we designed a combined HDMI/custom overlay that includes our IP

Error: KeyboardInterrupt

In [1]:
## imports for custom overlay
from pynq import Overlay
import numpy as np
## imports for baseoverlay HDMI
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import numpy as np
import asyncio
## Import for DMA alloc
from pynq import allocate



In [2]:
PIXEL_RGB?

In [24]:
## BASE OVERLAY

base = BaseOverlay("base.bit")
vdma = base.video.axi_vdma

# Set HDMI in and out vars
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start()
hdmi_out.start()

In [25]:
# read a frame, allocate a buffer to hold it
frame = hdmi_in.readframe()

# in_buffer = allocate(shape=frame.shape, dtype=np.uint8)
# out_buffer = allocate(shape=frame.shape, dtype=np.uint8)
in_buffer = allocate(shape=720*1280*3, dtype=np.uint8)
out_buffer = allocate(shape=720*1280*3, dtype=np.uint8)

in_buffer[:] = np.ndarray.flatten(frame)
hdmi_out.writeframe(in_buffer)
print(len(in_buffer))
# print(len(in_buffer[0]))
# print(len(in_buffer[0][0]))

2764800


In [26]:
hdmi_out.close()
hdmi_in.close()

In [27]:
## CUSTOM OVERLAY
# Then do the stream overlay for the processing
stream_overlay = Overlay('passthru.bit')
dma = stream_overlay.redDMA.axi_dma_0
resolution = stream_overlay.redDMA.red_shift_0

In [28]:
resolution.write(0x10, 720)
resolution.write(0x18, 1280)

In [19]:
# send input buffer and read the output
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [29]:
run_kernel()


In [30]:
## BASE OVERLAY AGAIN

base = BaseOverlay("base.bit")
vdma = base.video.axi_vdma

# Set HDMI in and out vars
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start()
hdmi_out.start()

In [32]:
hdmi_out.writeframe(in_buffer)

In [33]:
hdmi_out.writeframe(out_buffer)

In [34]:
hdmi_out.close()
hdmi_in.close()